<H1> PlayTones </H1> <br> 
Look for a way to play tones using portaudio. 
<hr> 
<H2>Modification history </H2>
<table>
    <tr>
        <th> Modified </th>
        <th> BY </th>
        <th> Reason </th>
    </tr>
    <tr>
        <td> 18-Feb-25</td>
        <td> CBL </td>
        <td> Original </td>
    </tr>
    <tr>
        <td> 24-Mar-25</td>
        <td> CBL </td>
        <td> Added in drive capability, drive the Modal 2000E device </td>
    </tr>
    <tr>
        <td> 19-May-25</td>
        <td> CBL </td>
        <td> Play Riken Data </td>
    </tr>
</table>

<hr> 
<H2> References </H2> 
<a href="https://people.csail.mit.edu/hubert/pyaudio/docs/">  Documentation </a> <br> 
<a href="https://stackoverflow.com/questions/40704026/voice-recording-using-pyaudio"> example recording</a> <br> 
<a href="https://github.com/jleb/pyaudio/blob/master/test/record.py"> example 2 </a><br> 
<a href="https://stackoverflow.com/questions/4623572/how-do-i-get-a-list-of-my-devices-audio-sample-rates-using-pyaudio-or-portaudio"> Get Sound device properties. </a><br> 
<a href="https://python-sounddevice.readthedocs.io/en/0.5.1/installation.html"> Sounddevice documentation</a>
<H2> Pre-requisites</H2> 
numpy<br>
matplotlib<br>
pyaudio<br>
scipy<br>
sounddevice - which returns data in numpy arrays!<br> 
<H3> Modal 2000E information</H3>
<a href="https://www.modalshop.com/vibration-test/products/vibration-test-shakers/2-lbf-mini-inertial-shaker"> Modal 2000E</a> <br> 
Force rating 9N <br> 
Max Frequency 3000Hz <br> 
Max stroke 8.9mm <br> 
Shaker Model: 2002E <br> 
Input voltage to Drive Amplifier: 0-1 V<br> 
<hr>

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as konst
import pyaudio
import wave
import sounddevice as sd
import scipy  as sp

In [5]:
def ConvertWaveform(yin):
    """
    @param yin - mono input, analog values {0:1}
    @return vector of L,R alternating pairs. Same output on both. Assume 16 bit resolution as output. 
    """
    y  = yin * 65535          # make 16 bit max value. 
    IY = y.astype(np.int16) # convert to integer
    # allocate output array
    N = len(IY)
    rv = np.zeros(2*N+1, dtype=np.int16)
    for i in range(0,N):
        rv[2*i] = IY[i]
        rv[2*i+1] = IY[i]
    return rv

In [6]:
def PlayWaveform(DeviceIndex, Waveform, SampleRate):
    """
    @brief - Play provided waveform to device, ostensibly the shaker
    @param DeviceIndex - choose output device
    @param Waveform - waveform to play. 
    """
    print("Play waveform, SampleRate:", SampleRate)
    # Instantiate PyAudio and initialize PortAudio system resources (1)
    p = pyaudio.PyAudio()
    devinfo = p.get_device_info_by_index(DeviceIndex)  # Or whatever device you care about.
    print("Devinfo: ", devinfo)

    # Open stream (2)
    stream = p.open(format=p.get_format_from_width(2),
                    channels=2,                       # assume a stereo output
                    rate=SampleRate,
                    output_device_index = DeviceIndex,
                    output=True)

    # Play samples from the wave file (3)
    stream.write(Waveform)

    # Close stream (4)
    stream.close()

    # Release PortAudio system resources (5)
    p.terminate()


In [7]:
def ShowDevices():
    p = pyaudio.PyAudio()
    info = p.get_host_api_info_by_index(0)
    numdevices = info.get('deviceCount')

    print('Number of devices: ', numdevices)

    for i in range(0, numdevices):
        in_channels = p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')
        out_channels = p.get_device_info_by_host_api_device_index(0, i).get('maxOutputChannels')
        print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'), " inputs: ", in_channels,
             " outputs: ", out_channels)


In [8]:
def ScanRates(device):
    """
    Using sounddevice
    @param device - integer index to device
    """
    samplerates = 16000,32000, 44100, 48000, 96000, 128000

    supported_samplerates = []
    for fs in samplerates:
        try:
            sd.check_output_settings(device=device, samplerate=fs)
        except Exception as e:
            print('Exception: ',fs, e)
        else:
            supported_samplerates.append(fs)
    print(supported_samplerates)


In [17]:
def CheckInfoOutput(DeviceIndex=1):
    """
    pyaudio version
    @param DeviceIndex - index into which device we should select. 
    pyaudio version. 
    """
    # sample rates we want to check. 
    samplerates = 1024, 2048, 4096, 8192, 16000,32000, 44100, 48000, 96000, 128000
    
    supported_samplerates = []
    
    p = pyaudio.PyAudio()
    devinfo = p.get_device_info_by_index(DeviceIndex)  # Or whatever device you care about.
    print("Devinfo: ", devinfo)
    input_N  = devinfo['maxInputChannels']
    output_N = devinfo['maxOutputChannels']
    print('Max output: ', output_N)
    if (output_N <1):
        print('No output device at this index.')
        return
    #
    # The macbook documentation says that the internal ADC can only support
    # 
    for fs in samplerates:
        if p.is_format_supported( fs,  # Sample rate
                                 output_device=devinfo['index'],
                                 output_channels=devinfo['maxOutputChannels'],
                                 output_format=pyaudio.paInt16):
            supported_samplerates.append(fs)

    print(supported_samplerates)


In [10]:
ShowDevices()

Number of devices:  7
Input Device id  0  -  G257HU  inputs:  0  outputs:  2
Input Device id  1  -  HD Pro Webcam C920  inputs:  2  outputs:  0
Input Device id  2  -  Widex SoundConnect  inputs:  0  outputs:  2
Input Device id  3  -  MacBook Pro Microphone  inputs:  1  outputs:  0
Input Device id  4  -  MacBook Pro Speakers  inputs:  0  outputs:  2
Input Device id  5  -  Microsoft Teams Audio  inputs:  1  outputs:  1
Input Device id  6  -  ZoomAudioDevice  inputs:  2  outputs:  2


In [6]:
#
# Make a waveform
# 
F0 = 400  # Hz
W  = 2.0 * konst.pi*F0
SampleRate = 48000
NSeconds = 5*60
t = np.arange(0,NSeconds*SampleRate)/SampleRate
y = 0.25*(1.0 + np.sin(W*t))
#plt.plot(y)
#PlotWelch(y, 1.0, SampleRate)
# turn into a short
# Scale to short
#plt.plot(IY[:400])

In [ ]:
IY = ConvertWaveform(y)
#plt.plot(IY[0:400])
# Mac speakers 4
# 
PlayWaveform(0, IY, SampleRate)

Play waveform, SampleRate: 48000
Devinfo:  {'index': 0, 'structVersion': 2, 'name': 'G257HU', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.01, 'defaultLowOutputLatency': 0.009833333333333333, 'defaultHighInputLatency': 0.1, 'defaultHighOutputLatency': 0.019166666666666665, 'defaultSampleRate': 48000.0}


In [35]:
PlayWaveform(1, IY, SampleRate)

Play waveform, SampleRate: 48000
Devinfo:  {'index': 1, 'structVersion': 2, 'name': 'Cable Creation', 'hostApi': 0, 'maxInputChannels': 1, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.01, 'defaultLowOutputLatency': 0.0038541666666666668, 'defaultHighInputLatency': 0.1, 'defaultHighOutputLatency': 0.0131875, 'defaultSampleRate': 48000.0}


In [16]:
CheckInfoOutput(0)

Devinfo:  {'index': 0, 'structVersion': 2, 'name': 'G257HU', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.01, 'defaultLowOutputLatency': 0.009833333333333333, 'defaultHighInputLatency': 0.1, 'defaultHighOutputLatency': 0.019166666666666665, 'defaultSampleRate': 48000.0}
Max output:  2
[1024, 2048, 4096, 8192, 16000, 32000, 44100, 48000, 96000, 128000]
